# Imports

In [24]:
# Imports
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import pywt

In [25]:
from IPython.display import clear_output

# Getting csv files

In [26]:
# Geting csv files
list_of_files = os.listdir("./")
csv_files = []

for _file in list_of_files:
    if _file.endswith(".csv"):
        csv_files.append(_file)

csv_files

['5_When_rebuilding.csv',
 '1_After_reconstruction.csv',
 '1_When_rebuilding.csv',
 '2_Before_renovation.csv',
 '6_After_reconstruction.csv',
 '2_When_rebuilding.csv',
 '4_Before_renovation.csv',
 '5_After_reconstruction.csv',
 '3_When_rebuilding.csv',
 '3_After_reconstruction.csv',
 '6_Before_renovation.csv',
 '4_After_reconstruction.csv',
 '1_Before_renovation.csv',
 '3_Before_renovation.csv',
 '6_When_rebuilding.csv',
 '2_After_reconstruction.csv',
 '5_Before_renovation.csv',
 '4_When_rebuilding.csv']

# Functions

In [27]:
def slice_signal_by_10_seconds(signal, sampling_rate):
    # Calculate the number of samples corresponding to 10 seconds
    ten_sec_samples = 10 * sampling_rate
    
    # Slice the signal into chunks of 10 seconds
    signal_chunks = [signal[i:i + ten_sec_samples] for i in range(0, len(signal), ten_sec_samples)]
    
    return signal_chunks

In [28]:
def extract_train_out_signal_chunks(signal, sampling_rate):
    '''
    Returns: The first 10-second signal and last 30-second signal which
                sliced by 10 seconds
    '''
    # Calculate the number of samples corresponding to 10 seconds
    ten_sec_samples = 10 * sampling_rate
    
    # Calculate the number of samples corresponding to 30 seconds
    thirty_sec_samples = 30 * sampling_rate
    
    # Get the first 10 seconds of the signal
    first_10_sec = signal[:ten_sec_samples]
    
    # Get the last 30 seconds of the signal
    last_30_sec = signal[-thirty_sec_samples:]
    
    # Split the last 30 seconds into chunks of 10 seconds
    chunks_last_30_sec = slice_signal_by_10_seconds(last_30_sec, sampling_rate)
    
    return first_10_sec, chunks_last_30_sec